In [12]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125613")
exp = Experiment(workspace=ws, name="myexperiment")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125613
Azure region: southcentralus
Subscription id: da775cb9-9ca6-4943-ad21-26dfa99526fc
Resource group: aml-quickstarts-125613


In [13]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

cpu_cluster_name = "azure-compute"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Cluster Found!.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)

Cluster Found!.


In [14]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive.parameter_expressions import choice

# Specify parameter sampler
### YOUR CODE HERE ###

ps = RandomParameterSampling({
    "C" : uniform(0.1,1),
    "max_iter" : choice(50,100,150,200) })

# Specify a Policy
### YOUR CODE HERE ###

policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###

est = SKLearn(source_directory = './training', compute_target='cpu_cluster', entry_script='final.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
 ### YOUR CODE HERE ###

hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy = policy,
                                primary_metric_name="Accuracy",
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=10,
                                max_concurrent_runs=4)

In [15]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

experiment = Experiment(workspace=ws, name="Project_1")
hyperdrive_run = experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_6aab852f-3c63-4ddf-9ba7-f7e98bb799c8
Web View: https://ml.azure.com/experiments/Project_1/runs/HD_6aab852f-3c63-4ddf-9ba7-f7e98bb799c8?wsid=/subscriptions/da775cb9-9ca6-4943-ad21-26dfa99526fc/resourcegroups/aml-quickstarts-125613/workspaces/quick-starts-ws-125613

Execution Summary
RunId: HD_6aab852f-3c63-4ddf-9ba7-f7e98bb799c8
Web View: https://ml.azure.com/experiments/Project_1/runs/HD_6aab852f-3c63-4ddf-9ba7-f7e98bb799c8?wsid=/subscriptions/da775cb9-9ca6-4943-ad21-26dfa99526fc/resourcegroups/aml-quickstarts-125613/workspaces/quick-starts-ws-125613


ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Microsoft.HyperDrive.Scheduler.AML.Client.AMLClientException - Message: \"{\"Error\":{\"Code\":\"UserError\",\"Message\":\"Unknown compute target 'cpu_cluster'.\",\"MessageFormat\":\"Unknown compute target '{TargetName}'.\",\"MessageParameters\":{\"TargetName\":\"cpu_cluster\"},\"ReferenceCode\":null,\"DetailsUri\":null,\"Target\":null,\"Details\":[],\"InnerError\":{\"Code\":\"BadArgument\",\"InnerError\":{\"Code\":\"UnknownTargetType\",\"InnerError\":null}},\"DebugInfo\":null},\"Correlation\":{\"operation\":\"e24e1ee45ca35645b10da66f10680552\",\"request\":\"b502afce4168f842\"},\"Environment\":\"southcentralus\",\"Location\":\"southcentralus\",\"Time\":\"2020-11-11T06:28:52.9807159+00:00\"}\", StatusCode: BadRequest, StackTrace: , InnerException: null",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:0

In [9]:


import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:


from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:


from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:


from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [ ]:


# Submit your automl run

### YOUR CODE HERE ###

In [ ]:


# Retrieve and save your best automl model.

### YOUR CODE HERE ###